In [1]:
from codebase.classes_data import Data
import argparse
import numpy as np
from codebase.file_utils import save_obj, load_obj, make_folder, path_backslash
from codebase.ibis import exp_and_normalise
from run_smc2 import run_smc2
from scipy.special import logsumexp
from pdb import set_trace


In [2]:
existing_directory = None
task_handle = 'smc2'
gen_model = False

if existing_directory is None:
    log_dir = make_folder(task_handle)
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)





Creating new directory: ./log/20210411_205314_smc2/


In [3]:
# generate data
exp_data = Data(name=task_handle, model_num=1, size=100, random_seed=2)


exp_data.generate()
save_obj(exp_data, "data", log_dir)
model_num = 3


In [4]:
%load_ext autotime

smc2 = run_smc2(exp_data, model_num, 200, 100, gen_model, log_dir)


  3%|▎         | 3/100 [03:36<1:53:30, 70.21s/it]/home/konstantinos/anaconda3/envs/pystan-dev/lib/python3.7/site-packages/numpy/lib/arraysetops.py:314: RuntimeWarning: invalid value encountered in not_equal
  mask[1:] = aux[1:] != aux[:-1]
  7%|▋         | 7/100 [15:34<3:23:51, 131.52s/it]

KeyboardInterrupt: 

time: 20min 54s (started: 2021-04-11 20:53:14 -04:00)


##

In [ ]:
particles.extract_particles_in_numpy_array('alpha')

In [ ]:
corrs = load_obj('jitter_corrs', log_dir)
corrs['beta']

In [ ]:
particles.get_acceptance_rate_for_particle_m(0)[particles.get_threshold_ess_indicator()]

In [ ]:
particles.get_acceptance_rate_for_particle_m(2)[particles.get_threshold_ess_indicator()]

In [ ]:
particles.get_threshold_ess_indicator()

In [ ]:
particles.resample_particles_bundles()

# particles.gather_latent_variables_up_to_t(
#     t+1, 
#     exp_data.get_stan_data_upto_t(t+1)
# )

# particles.jitter_bundles_and_pick_one(exp_data.get_stan_data_upto_t(t + 1))
# particles.check_latent_particles_are_distinct()

# particles.jitter(exp_data.get_stan_data_upto_t(t + 1))

In [ ]:
ps_smc2 = dict()
for name in particles.param_names:
    ps_smc2[name] = particles.extract_particles_in_numpy_array(name)


In [ ]:
ps_smc2 = post_process_sign(ps_smc2)


## Plot

In [ ]:
param = 'beta'
df = get_post_df(ps_smc2[param]) 
df['source'] = 'smc2'
plot_density(df, height=100)

In [ ]:
param = 'alpha'
df = get_post_df(ps_smc2[param]) 
df['source'] = 'smc2'
plot_density(df, height=100)

## Plot MCMC samples

In [ ]:
param = 'beta'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

In [ ]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [ ]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


In [ ]:
load_obj('marg_lklhd', log_dir)